In [11]:
import pandas as pd 
import numpy as np 
import math
import random 

In [12]:
def fitness(a):

    print("Guaranteed ")
    fit = int(0)
    for i in range(0, len(a)-1):
        fit = fit + adj[a[i]][a[i+1]]
    fit = fit + adj[a[len(a)-1]][a[0]] 
    
    return fit 

In [13]:
class Ant:
    def __init__(self, myID, cur):
        self.visited = [] 
        self.id = myID
        self.current = int(cur) 
        self.fitness = 0

In [14]:
class Solver:
    def __init__(self, n, l, alpha, beta, rEvap, pheroC, ages):
        self.ages = ages
        self.n = n
        self.l = l #number of ants
        self.alpha = alpha 
        self.beta = beta
        self.rEvap = rEvap 
        self.pheroC = pheroC 

        self.phero = [[1 for i in range(n)] for j in range(n)] 
        self.ants = []

        self.ans = []
        self.ansFitness = 9999999999
        for i in range(n):
            for j in range(n):
                if i != j:
                    self.phero[i][j] = 1
    def solve(self):
        self.history = []
        for _ in range(self.ages):
            self.ants = []
            if _%1 == 0 and _ != 0:
                self.history.append(self.ansFitness)
            for i in range(self.l):
                self.firstPos = random.choice(range(self.n))
                self.newAnt = Ant(myID = i, cur = self.firstPos)
                self.newAnt.visited.append(self.firstPos)
                self.ants.append(self.newAnt)
                
            for i in range(self.n-1): #Every ant must move n-1 time
                
                for j in range(self.l):
                    self.Move(self.ants[j])
            
            for j in range(self.l):
                self.ants[j].fitness+= adj[self.ants[j].current][self.ants[j].visited[0]] #back to the first one
            for i in range(self.l):
                if self.ants[i].fitness < self.ansFitness:
                    self.ansFitness = self.ants[i].fitness
                    self.ans = self.ants[i].visited
            self.Evaporate()

    def goTo(self, A):
        self.Sigma = 0
        self.allowed = {i for i in range(self.n)}
        self.visTemp = {x for x in A.visited}
        self.allowed = self.allowed.difference(self.visTemp)
        self.allowed = list(self.allowed)
        for x in self.allowed:
            if adj[A.current][x] == 0:
                adj[A.current][x] = 99
            self.Sigma += self.phero[A.current][x] **self.alpha * ( (1 / adj[A.current][x] ) ** self.beta ) 
            
        self.weighted = [ (self.phero[A.current][x]**self.alpha * (1/adj[A.current][x])**self.beta)/self.Sigma  for x in self.allowed]
        #print(allowed)
        #print(weighted)
        
        self.chosen = random.choices(population=self.allowed, weights=self.weighted, k = 1)
        self.chosen = self.chosen[0]
        #print(chosen)
        return int(self.chosen)

    def Move(self, A):
        choose = self.goTo(A)
        self.phero[A.current][choose] += self.pheroC/adj[A.current][choose]
        self.phero[choose][A.current] = self.phero[A.current][choose]
        A.fitness += adj[A.current][choose]
        A.current = choose
        A.visited.append(choose)
    
    

    def Evaporate(self):
        for i in range(self.n):
            for j in range(i+1, self.n):
                self.phero[i][j] *= (1-self.rEvap)
                self.phero[i][j] = max(self.phero[i][j], 0.2)
                self.phero[j][i] = self.phero[i][j]
            

The Solver class represents the solver for the TSP using the ACO algorithm. In the constructor __init__, the necessary parameters such as the number of cities (n), number of ants (l), pheromone evaporation rate (rEvap), and pheromone constant (pheroC) are initialized. The pheromone matrix is also initialized, and ants are created with their initial positions.

The solve method performs the main optimization loop. It iterates over a given number of ages, where in each iteration, ants are created and moved through the cities. The ants choose the next city to visit based on the pheromone levels and distance information. After completing the tour, the fitness of each ant is calculated, and if a better solution is found, it is updated. The pheromone is evaporated, and the process continues until all iterations are completed.

The goTo method calculates the probabilities for the ant to move to each unvisited city based on the pheromone and distance information. The Move method chooses the next city for the ant based on the probabilities calculated by goTo and updates the pheromone levels and fitness accordingly. The Evaporate method reduces the pheromone levels with a given evaporation rate, ensuring a balance between exploration and exploitation.

Overall, the code sets up the necessary parameters, initializes ants, performs ant movement, updates pheromone levels, and continues the process for a given number of iterations, aiming to find an optimal solution for the TSP using the ACO algorithm.

In [15]:
def inputpr1002(path):
    def dist(i,j):
        res = math.sqrt((nodes[i][0]-nodes[j][0])**2+(nodes[i][1]-nodes[j][1])**2)
        return int(res)
    
    f = open(path)
    inp = f.read()
    f.close()

    nodes = [(float(e.split()[1]),float(e.split()[2])) for e in inp.split('\n')[6:-1]]

    mat = [[dist(i,j) for j in range(len(nodes))] for i in range(len(nodes))]
    return len(mat),mat


size, adj = inputpr1002('/content/pr1002.tsp')
instance1002 = Solver(1002, 10, 0.5, 5, 1/80, 5, 50)

In [24]:
f = open("/content/gr229.tsp")
inp = f.read()
f.close()
nodes = [(float(e.split()[1]),float(e.split()[2])) for e in inp.split('\n')[7:-2]]
adj = [[int(math.sqrt((nodes[i][0]-nodes[j][0])**2+(nodes[i][1]-nodes[j][1])**2)) for j in range(len(nodes))] for i in range(len(nodes))]
instance229 = Solver(229, 10, 0.5, 5, 1/80, 5, 50)

In [16]:
instance1002.solve()

In [22]:
print("Distance for pr1002.tsp input: ", instance1002.ansFitness)
print("List of cities: ", instance1002.ans)

Distance for pr1002.tsp input:  375269
List of cities:  [356, 357, 358, 359, 360, 350, 349, 348, 345, 346, 347, 389, 391, 390, 352, 351, 338, 339, 388, 392, 386, 385, 387, 393, 394, 383, 382, 384, 381, 379, 380, 377, 367, 366, 365, 364, 362, 361, 363, 658, 657, 654, 655, 656, 651, 652, 653, 642, 643, 650, 647, 645, 644, 648, 649, 646, 638, 639, 372, 371, 370, 369, 368, 376, 378, 411, 410, 409, 415, 413, 414, 416, 405, 406, 401, 402, 403, 287, 286, 285, 284, 283, 282, 281, 280, 276, 277, 279, 278, 112, 110, 109, 111, 113, 114, 115, 117, 105, 104, 103, 102, 101, 100, 99, 132, 133, 94, 95, 96, 97, 90, 91, 89, 92, 93, 76, 77, 78, 81, 87, 85, 88, 86, 84, 83, 82, 80, 79, 48, 40, 49, 39, 38, 37, 36, 34, 32, 60, 64, 58, 57, 59, 65, 66, 55, 54, 56, 52, 50, 51, 53, 71, 72, 70, 69, 68, 67, 11, 10, 9, 13, 12, 15, 16, 21, 22, 23, 24, 30, 29, 28, 25, 26, 27, 331, 332, 333, 326, 328, 327, 323, 324, 325, 330, 329, 314, 315, 318, 317, 319, 316, 310, 309, 311, 44, 43, 42, 41, 47, 46, 45, 307, 306, 305, 

In [25]:
instance229.solve()

In [26]:
print("Distance for pr1002.tsp input: ", instance229.ansFitness)
print("List of cities: ", instance229.ans)

Distance for pr1002.tsp input:  1963
List of cities:  [62, 60, 59, 61, 64, 58, 54, 53, 57, 52, 18, 10, 9, 7, 6, 4, 3, 5, 8, 2, 0, 1, 12, 13, 14, 21, 20, 19, 15, 11, 16, 17, 80, 81, 82, 56, 55, 23, 22, 83, 24, 84, 85, 88, 87, 78, 79, 89, 76, 75, 74, 73, 90, 91, 92, 93, 99, 100, 98, 97, 96, 95, 94, 36, 35, 34, 33, 32, 29, 30, 28, 31, 38, 39, 40, 179, 177, 181, 182, 184, 185, 186, 196, 197, 195, 194, 191, 192, 193, 190, 189, 188, 187, 48, 46, 47, 49, 183, 45, 41, 44, 43, 42, 163, 164, 172, 171, 162, 134, 131, 130, 129, 127, 126, 125, 110, 113, 116, 115, 117, 118, 119, 120, 122, 121, 114, 111, 105, 106, 104, 103, 107, 108, 109, 123, 124, 165, 37, 112, 128, 137, 133, 132, 170, 169, 168, 166, 167, 176, 178, 180, 174, 175, 198, 161, 160, 159, 158, 155, 153, 150, 151, 152, 135, 136, 139, 140, 142, 143, 144, 148, 147, 146, 145, 154, 149, 156, 157, 203, 211, 205, 206, 207, 208, 209, 210, 202, 201, 220, 221, 222, 199, 200, 219, 218, 215, 214, 213, 212, 204, 141, 138, 173, 69, 70, 71, 72, 68, 66, 